# Eliminate duplicates 

Goal of this notebook to to take the rows that have the same information for all columns (except `reporting period`) so that only the most recent reporting period is there. 

In [22]:
# ! pip install nltk

In [2]:
import numpy as np
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

from shared_utils import geography_utils

# import utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
import dla_utils

In [4]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

In [5]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

### Read in data 

In [6]:
df = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/AllOutcomesasof02-17-2023.xlsx"))

In [7]:
df.sample(5)

,dist_ppno,district,ppno,ea,project_id,funding_sources,on_system,outcome_desc,unit_desc,total,parent_ppnos,parent_ids,reporting_period,report_type
6386,03-1535A,3,1535A,NaN,0322000207,SCCP,N,Safety: Number of Fatalities,Number,9.38,NaN,NaN,FY 22/23 Q1,PROGRESS
12427,03-5101,3,5101,3F232,0321000123,SCCP,Y,Congestion Reduction: Change in Daily Vehicle Miles Travelled,VMT per Capita,29.92,NaN,NaN,FY 21/22 Q3,PROGRESS
16944,07-T0006,7,T0006,TC006,0719000121,TCEP,N,Reduction in Particulate Matter (PM2.5),Tons per year,0.2,NaN,NaN,FY 22/23 Q1,PROGRESS
2236,04-0462J,4,0462J,26470,0400000740,LPP-Comp,Y,Cost Effectiveness (only ‘Change’ required): Cost Benefit Ratio,Ratio,3.11,NaN,NaN,FY 20/21 Q4,PROGRESS
13797,07-5497,7,5497,NaN,0719000038,SCCP,N,Reduction in Particulate Matter (PM10),Tons per year,15.85,NaN,NaN,FY 21/22 Q3,PROGRESS


In [8]:
# df[['FY', 'Quarter']] = df['reporting_period'].str.split(' Q', 1, expand=True)

In [9]:
df

,dist_ppno,district,ppno,ea,project_id,funding_sources,on_system,outcome_desc,unit_desc,total,parent_ppnos,parent_ids,reporting_period,report_type
0,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 20/21 Q4,PROGRESS
1,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q1,PROGRESS
2,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q2,PROGRESS
3,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q3,PROGRESS
4,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17691,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Cargo Volume That Can Be Accommodated,# of Tons,96890613,NaN,NaN,FY 22/23 Q2,PROGRESS
17692,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Rail Volume That Can Be Accommodated,# of Containers,269141,NaN,NaN,FY 22/23 Q2,PROGRESS
17693,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Rail Volume That Can Be Accommodated,# of Trailers,1019,NaN,NaN,FY 22/23 Q2,PROGRESS
17694,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Truck Volume That Can Be Accommodated,# of Trucks,1295,NaN,NaN,FY 22/23 Q2,PROGRESS


### test with one outcome 

In [10]:
(df>>filter(_.outcome_desc == "Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2)"))>>group_by(_.outcome_desc, _.project_id)>>arrange(-_.reporting_period)

,dist_ppno,district,ppno,ea,project_id,funding_sources,on_system,outcome_desc,unit_desc,total,parent_ppnos,parent_ids,reporting_period,report_type
4773,11-1328,11,1328,NaN,1118000226,"ATP,LPP-Form",N,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,-8.94029,NaN,NaN,FY 22/23 Q2,PROGRESS
16740,04-T0003,4,T0003,TC0003,0018000297,TCEP,N,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,-9975,NaN,NaN,FY 22/23 Q2,PROGRESS
4,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
297,05-0073A,5,0073A,0C732,0512000228,"LPP-Comp,SCCP",Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
519,05-0073C,5,0073C,0C733,0518000116,"LPP-Comp,SCCP",Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,522289.125,NaN,NaN,FY 21/22 Q4,PROGRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16160,04-8273C,4,8273C,0A53T,0421000155,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,NaN,NaN,NaN,FY 20/21 Q4,PROGRESS
16318,75-9883,75,9883,NaN,NaN,TCEP,N,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,NaN,NaN,NaN,FY 20/21 Q4,PROGRESS
16489,75-LP003,75,LP003,NaN,0021000174,TCEP,N,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,NaN,NaN,NaN,FY 20/21 Q4,PROGRESS
17186,75-T0015,75,T0015,NaN,0021000173,TCEP,N,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,96539,NaN,NaN,FY 20/21 Q4,PROGRESS


#### add in project ID to test data to make sure they are all the same project

In [23]:
df_test = (df>>filter(_.outcome_desc == "Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2)", _.project_id=='0518000120'))

In [24]:
df_test>>arrange(-_.reporting_period)

,dist_ppno,district,ppno,ea,project_id,funding_sources,on_system,outcome_desc,unit_desc,total,parent_ppnos,parent_ids,reporting_period,report_type
4,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
3,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q3,PROGRESS
2,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q2,PROGRESS
1,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q1,PROGRESS
0,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 20/21 Q4,PROGRESS


In [25]:
df_test

,dist_ppno,district,ppno,ea,project_id,funding_sources,on_system,outcome_desc,unit_desc,total,parent_ppnos,parent_ids,reporting_period,report_type
0,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 20/21 Q4,PROGRESS
1,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q1,PROGRESS
2,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q2,PROGRESS
3,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q3,PROGRESS
4,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS


In [26]:
list(df_test)

['dist_ppno',
 'district',
 'ppno',
 'ea',
 'project_id',
 'funding_sources',
 'on_system',
 'outcome_desc',
 'unit_desc',
 'total',
 'parent_ppnos',
 'parent_ids',
 'reporting_period',
 'report_type']

In [27]:
df_test.drop_duplicates(subset=['dist_ppno', 'district', 'ppno', 'ea', 'project_id',
 'funding_sources', 'on_system', 'outcome_desc', 'unit_desc', 'total', 'parent_ppnos', 'parent_ids', 'report_type']
                        ,keep='last')


,dist_ppno,district,ppno,ea,project_id,funding_sources,on_system,outcome_desc,unit_desc,total,parent_ppnos,parent_ids,reporting_period,report_type
4,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS


### apply to the while df

In [28]:
df_drop1 = df.drop_duplicates(subset=['dist_ppno', 'district', 'ppno', 'ea', 'project_id',
 'funding_sources', 'on_system', 'outcome_desc', 'unit_desc', 'total', 'parent_ppnos', 'parent_ids', 'report_type']
                        ,keep='last')


In [29]:
len(df_drop1)

6240

In [30]:
df_drop1.drop_duplicates(keep='first')


,dist_ppno,district,ppno,ea,project_id,funding_sources,on_system,outcome_desc,unit_desc,total,parent_ppnos,parent_ids,reporting_period,report_type
4,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
9,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Monoxide (CO),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
14,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Nitrogen Oxides (NOx),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
23,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Particulate Matter,PM 10 Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
24,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Particulate Matter,PM 2.5 Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17691,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Cargo Volume That Can Be Accommodated,# of Tons,96890613,NaN,NaN,FY 22/23 Q2,PROGRESS
17692,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Rail Volume That Can Be Accommodated,# of Containers,269141,NaN,NaN,FY 22/23 Q2,PROGRESS
17693,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Rail Volume That Can Be Accommodated,# of Trailers,1019,NaN,NaN,FY 22/23 Q2,PROGRESS
17694,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Truck Volume That Can Be Accommodated,# of Trucks,1295,NaN,NaN,FY 22/23 Q2,PROGRESS


In [31]:
len(df_drop1)

6240

In [32]:
len(df)

17696

### add to a function

In [50]:
def get_most_recent_records(df,
                            date_col
                           ):
    ## get list of columns and remove the column where we can identify which record was the most recent
    list_of_cols = list(df)
    list_of_cols.remove(date_col) 
    
    #drop the duplicates
    new_df = df.drop_duplicates(subset=list_of_cols, keep='last')
        
    return new_df
                            
                            

In [52]:
get_most_recent_records(df, 'reporting_period')

,dist_ppno,district,ppno,ea,project_id,funding_sources,on_system,outcome_desc,unit_desc,total,parent_ppnos,parent_ids,reporting_period,report_type
4,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Dioxide (CO2),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
9,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Carbon Monoxide (CO),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
14,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Nitrogen Oxides (NOx),Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
23,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Particulate Matter,PM 10 Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
24,05-0057D,5,0057D,31601,0518000120,TCEP,Y,Air Quality & GHG (only ‘Change’ required): Particulate Matter,PM 2.5 Tons,0,NaN,NaN,FY 21/22 Q4,PROGRESS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17691,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Cargo Volume That Can Be Accommodated,# of Tons,96890613,NaN,NaN,FY 22/23 Q2,PROGRESS
17692,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Rail Volume That Can Be Accommodated,# of Containers,269141,NaN,NaN,FY 22/23 Q2,PROGRESS
17693,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Rail Volume That Can Be Accommodated,# of Trailers,1019,NaN,NaN,FY 22/23 Q2,PROGRESS
17694,75-T0019,75,T0019,NaN,0021000176,TCEP,N,Throughput: Change in Truck Volume That Can Be Accommodated,# of Trucks,1295,NaN,NaN,FY 22/23 Q2,PROGRESS
